In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end


using MorphoMol
using JLD2
using LinearAlgebra
using Rotations
using Printf

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`
Precompiling MorphoMol
  ✓ MorphoMol
  1 dependency successfully precompiled in 4 seconds. 243 already precompiled.


In [5]:
function get_aggregate_simulation_evaluation(folder)
    mol_type = "6r7m"
    assembly_successes = 0
    total_simulations = 0
    id_index = 1
    steps_from_init_to_success = Vector{Int}([])
    steps_from_last_max_to_success = Vector{Int}([])
    acceptance_rates = Vector{Float64}([])

    for file in readdir(folder)
        if split(file, ".")[end] == "jld2"
            try
                @load "$folder$file" input output

                E_min_index = argmin(output["Es"])
                E_max_index = argmax(output["Es"])

                # Crude test to exclude runs that remained dispersed
                if maximum(output["Es"]) - minimum(output["Es"]) < 10.0
                    println("E_diff = $(output["Es"][E_max_index] - output["Es"][E_min_index])")
                    continue
                end

                exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
                exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
                theta = MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, output["states"][argmin(output["Es"])])
                push!(acceptance_rates, output["αs"][end])

                E_max = maximum(output["Es"])
                last_E_disp_index = E_max_index > E_min_index ? 1 : findall(x->x≈E_max, output["Es"][1:E_min_index])[end]
    
                total_simulations += 1
                if theta < 2.0
                    assembly_successes += 1
                    push!(steps_from_init_to_success, E_min_index)
                    push!(steps_from_last_max_to_success, E_min_index - last_E_disp_index)
                end
            catch e
                println("Error in $file")
            end
        end
    end
    return total_simulations, assembly_successes/total_simulations, sum(steps_from_init_to_success)/length(steps_from_init_to_success), sum(steps_from_last_max_to_success)/length(steps_from_init_to_success), sum(acceptance_rates) / length(acceptance_rates)
end

get_aggregate_simulation_evaluation (generic function with 1 method)

In [6]:
#folder = "../../Data/collected_simulation_results/rwm_ma_2_6r7m/"
folder = "../../Data/hpc_out/rwm_ma_2_6r7m/"
ts, sr, sfits, sfmts, aar = get_aggregate_simulation_evaluation(folder)
println("Morphometric Approach")
@printf("Number of simulations: %.0f, Success ratio: %.3f\nSteps from init to success: %.0f, Steps from last max to success: %.0f\nAverage acceptance rate: %.2f", ts, sr, sfits, sfmts, aar)

Morphometric Approach
Number of simulations: 75, Success ratio: 0.107
Steps from init to success: 6116, Steps from last max to success: 2327
Average acceptance rate: 0.19

In [7]:
folder = "../../Data/hpc_out/rwm_wp_2_6r7m/"
ts, sr, sfits, sfmts, aar = get_aggregate_simulation_evaluation(folder)
println("Morphometric Approach with Persistence")
@printf("Number of simulations: %.0f, Success ratio: %.3f\nSteps from init to success: %.0f, Steps from last max to success: %.0f\nAverage acceptance rate: %.2f", ts, sr, sfits, sfmts, aar)

Morphometric Approach with Persistence
Number of simulations: 95, Success ratio: 0.137
Steps from init to success: 5124, Steps from last max to success: 4037
Average acceptance rate: 0.23

In [9]:
folder = "../../Data/hpc_out/rwm_op_2_6r7m/"
ts, sr, sfits, sfmts, aar = get_aggregate_simulation_evaluation(folder)
println("Only Persistence")
@printf("Number of simulations: %.0f, Success ratio: %.3f\nSteps from init to success: %.0f, Steps from last max to success: %.0f\nAverage acceptance rate: %.2f", ts, sr, sfits, sfmts, aar)

Only Persistence
Number of simulations: 92, Success ratio: 0.011
Steps from init to success: 15773, Steps from last max to success: 15772
Average acceptance rate: 0.28

In [11]:
folder = "../../Data/hpc_out/rwm_wip_2_6r7m/"
ts, sr, sfits, sfmts = get_aggregate_simulation_evaluation(folder)
println("Morphometric Approach with Interface Persistence")
@printf("Number of simulations: %.0f, Success ratio: %.3f, Steps from init to success: %.0f, Steps from last max to success: %.0f\n", ts, sr, sfits, sfmts)

E_diff = 0.17588891449122457
E_diff = 2.309947250445475
E_diff = 9.081121571569383
E_diff = 1.2891706372483895
E_diff = 0.18905542237007467
E_diff = 0.05964941272395663
E_diff = 0.183649390561186
E_diff = 1.146658331655999
E_diff = 0.16609082444483647
E_diff = 0.19282971879874822
E_diff = 4.020013015486256
E_diff = 0.17980169574184401
E_diff = 0.10223419732392358
E_diff = 0.17684290282159054
E_diff = 0.25411825279297773
E_diff = 0.1740903117279231
E_diff = 0.16827574921080668
E_diff = 8.071846033895781
E_diff = 0.22411166241727187
E_diff = 6.657116680400577
E_diff = 0.19947445879915904
E_diff = 8.735221106200697
E_diff = 0.23225610318331746
E_diff = 0.23643626219200087
E_diff = 0.2577421406404028
E_diff = 0.22874577008042252
E_diff = 0.19230514722221415
E_diff = 8.175936395649842
E_diff = 0.14797095430003537
E_diff = 0.13818695597001351
E_diff = 0.10445134579458681
E_diff = 0.24065456361859106
E_diff = 0.3018426446687954
E_diff = 3.6959893571183784
E_diff = 0.13236933948610385
E_diff =